In [296]:
import pandas as pd
import statistics as stats
import numpy as np

pattern_range = 10 # minimum number of data points that defines a trend
max_trend_points = 150 # maximum mnumber of data points in a trend
current_index = 0
x_axis = 'TEST_PERIOD_FORMATTED'
y_axis = 'ATT_TP_UL_MV'
trend_type = "TREND_TYPE"
file_name = 'amCharts Austin.csv'

In [297]:
data = pd.read_csv(file_name)
df = pd.DataFrame()
df[x_axis] = pd.to_datetime(data[x_axis])
df[y_axis] = data[y_axis]
df = df.dropna().reset_index(drop=True)
df[trend_type] = np.nan
df.head()

,TEST_PERIOD_FORMATTED,ATT_TP_UL_MV,TREND_TYPE
0,2019-07-02,9.91,NaN
1,2019-07-03,9.47,NaN
2,2019-07-04,11.80,NaN
3,2019-07-05,10.73,NaN
4,2019-07-06,11.88,NaN


In [298]:
current_trend = [] # list of datapoints part of current trend

# for displaying on graph
anomalies = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
anomalies[x_axis] = df[x_axis]
shifts = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
shifts[x_axis] = df[x_axis]
trends = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
trends[x_axis] = df[x_axis]

# builds initial trend from pattern_range number of datapoints
for current_index in range(pattern_range):
  current_trend.append(df[y_axis][current_index])

In [299]:
for current_index in range(pattern_range - 1, len(df[y_axis])):
  datapoint = df[y_axis][current_index]
  old_mean = stats.mean(current_trend)
  strdev = stats.stdev(current_trend, old_mean)

  # for trends data points on graph
  if (current_index + 1) % (pattern_range/2) == 0:
    trends.iat[current_index, 1] = old_mean
  
  # keeps the current trend list with recent points determined by max_trend_points value
  if len(current_trend) >= max_trend_points:
    current_trend.pop(0)

  # check if new datapoint is anomaly / trend (3 stdev out)
  if datapoint >= strdev * 3 + old_mean or datapoint <= old_mean - strdev * 3:
    temp_trend = []
    for i in range(pattern_range):
      if (current_index + i < len(df[y_axis])):
        temp_trend.append(df[y_axis][current_index + i])
    new_mean = stats.mean(temp_trend)

    if len(temp_trend) == pattern_range and (new_mean >= strdev * 3 + old_mean or new_mean <= old_mean - strdev * 3): # trend shift
      print("index: " + str(current_index) + " shift: " + str(df[x_axis][current_index]))
      if old_mean < new_mean:
        df[trend_type][current_index] = "shift up"
      elif old_mean > new_mean:
        df[trend_type][current_index] = "shift down"

      shifts.iat[current_index, 1] = datapoint
      current_trend.clear()
      current_trend.extend(temp_trend)
      current_index += pattern_range
    else: # anomaly
      print("index: " + str(current_index) + " anomaly: " + str(df[x_axis][current_index]))
      df[trend_type][current_index] = "anomaly"
      anomalies.iat[current_index, 1] = datapoint
  else:
    current_trend.append(datapoint)

index: 56 anomaly: 2019-08-27 00:00:00
index: 57 anomaly: 2019-08-28 00:00:00
index: 116 anomaly: 2019-10-26 00:00:00
index: 125 anomaly: 2019-11-04 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



index: 264 anomaly: 2020-03-25 00:00:00
index: 266 anomaly: 2020-03-27 00:00:00
index: 455 anomaly: 2020-10-04 00:00:00
index: 475 anomaly: 2020-10-25 00:00:00
index: 476 anomaly: 2020-10-26 00:00:00
index: 477 anomaly: 2020-10-27 00:00:00
index: 481 anomaly: 2020-10-31 00:00:00
index: 484 anomaly: 2020-11-03 00:00:00
index: 493 anomaly: 2020-11-12 00:00:00
index: 496 anomaly: 2020-11-15 00:00:00
index: 501 anomaly: 2020-11-20 00:00:00
index: 588 anomaly: 2021-02-15 00:00:00
index: 589 anomaly: 2021-02-16 00:00:00
index: 590 anomaly: 2021-02-17 00:00:00
index: 644 anomaly: 2021-04-15 00:00:00
index: 806 anomaly: 2021-09-26 00:00:00
index: 895 anomaly: 2021-12-24 00:00:00
index: 922 anomaly: 2022-01-20 00:00:00
index: 931 anomaly: 2022-01-29 00:00:00
index: 950 anomaly: 2022-02-17 00:00:00
index: 981 anomaly: 2022-03-20 00:00:00
index: 994 anomaly: 2022-04-02 00:00:00
index: 1001 anomaly: 2022-04-09 00:00:00
index: 1002 anomaly: 2022-04-10 00:00:00
index: 1015 anomaly: 2022-04-23 00:00:

In [300]:
def returnName(mean, first_mean, leniency): # returns trend name based on given means
  if mean + mean * leniency >= first_mean and mean - mean * leniency <= first_mean:
    return "flat"
  elif mean > first_mean:
    return "up"
  elif mean < first_mean:
    return "down"

# trend identification
first_mean = df[y_axis][0]
i = pattern_range/2 - 1
leniency = 0.001 # leniency for considering means to be equal, 0.1%
name = ""
while i + pattern_range/2 <= len(trends[y_axis]):
  mean = trends[y_axis][i]
  name = returnName(mean, first_mean, leniency)
  j = 0
  if not pd.isnull(mean):
    j = i - (pattern_range/2 - 1)
  else:
    i += pattern_range/2
    mean = trends[y_axis][i]
    name = returnName(mean, first_mean, leniency)
  while j <= i:
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

  first_mean = mean
  i += pattern_range/2

# trend identification for last values
if j < len(trends[y_axis]):
  mean = df[y_axis][len(df[y_axis]) - 1]
  name = returnName(mean, first_mean, leniency)
  while j < len(trends[y_axis]):
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

In [301]:
scale = 0
for i in range(len(df[y_axis])):
  print(str(df[x_axis][i + scale]) + " type: " + str(df[trend_type][i + scale]))

2019-07-02 00:00:00 type: up
2019-07-03 00:00:00 type: up
2019-07-04 00:00:00 type: up
2019-07-05 00:00:00 type: up
2019-07-06 00:00:00 type: up
2019-07-07 00:00:00 type: up
2019-07-08 00:00:00 type: up
2019-07-09 00:00:00 type: up
2019-07-10 00:00:00 type: up
2019-07-11 00:00:00 type: up
2019-07-12 00:00:00 type: down
2019-07-13 00:00:00 type: down
2019-07-14 00:00:00 type: down
2019-07-15 00:00:00 type: down
2019-07-16 00:00:00 type: down
2019-07-17 00:00:00 type: down
2019-07-18 00:00:00 type: down
2019-07-19 00:00:00 type: down
2019-07-20 00:00:00 type: down
2019-07-21 00:00:00 type: down
2019-07-22 00:00:00 type: up
2019-07-23 00:00:00 type: up
2019-07-24 00:00:00 type: up
2019-07-25 00:00:00 type: up
2019-07-26 00:00:00 type: up
2019-07-27 00:00:00 type: up
2019-07-28 00:00:00 type: up
2019-07-29 00:00:00 type: up
2019-07-30 00:00:00 type: up
2019-07-31 00:00:00 type: up
2019-08-01 00:00:00 type: up
2019-08-02 00:00:00 type: up
2019-08-03 00:00:00 type: up
2019-08-04 00:00:00 typ

In [302]:
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import math

fig = go.Figure()

if y_axis == 'ATT_TP_UL_MV':
  fig.add_trace(go.Scatter(x=df.TEST_PERIOD_FORMATTED, y=df.ATT_TP_UL_MV, 
                          mode='lines', name=y_axis))
  fig.add_trace(go.Scatter(x=anomalies.TEST_PERIOD_FORMATTED, y=anomalies.ATT_TP_UL_MV,
                          mode='markers', name='Anomaly'))
  fig.add_trace(go.Scatter(x=shifts.TEST_PERIOD_FORMATTED, y=shifts.ATT_TP_UL_MV,
                          mode='markers', name='Shift'))
  fig.add_trace(go.Scatter(x=trends.TEST_PERIOD_FORMATTED, y=trends.ATT_TP_UL_MV,
                          mode='markers', name='Trend'))
else:
  fig.add_trace(go.Scatter(x=df.TEST_PERIOD_FORMATTED, y=df.ATT_TP_DL_MV, 
                          mode='lines', name=y_axis))
  fig.add_trace(go.Scatter(x=anomalies.TEST_PERIOD_FORMATTED, y=anomalies.ATT_TP_DL_MV,
                          mode='markers', name='Anomaly'))
  fig.add_trace(go.Scatter(x=shifts.TEST_PERIOD_FORMATTED, y=shifts.ATT_TP_DL_MV,
                          mode='markers', name='Shift'))
  fig.add_trace(go.Scatter(x=trends.TEST_PERIOD_FORMATTED, y=trends.ATT_TP_DL_MV,
                          mode='markers', name='Trend'))
  
fig.update_layout(showlegend=True, title={
        'text': file_name[9:len(file_name) - 4],
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()